# V4 — 02 Backend Walkthrough (Gemini + Pinecone + Prompt)
Este notebook sirve para **probar y depurar** el flujo del backend antes de correr FastAPI:
1) Embedding de la pregunta (Gemini)
2) Retrieval (Pinecone)
3) Construcción del prompt con evidencia
4) Respuesta (Gemini)

Recomendación: crea un `.env` en la raíz del proyecto (junto a `requirements.txt`) y pon tus keys ahí.


## 0) Preparación (ruta del proyecto + cargar settings)

In [ ]:
import sys
from pathlib import Path

# Ajusta si estás ejecutando el notebook desde otro directorio
PROJECT_ROOT = Path("..").resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from natubot_core.settings import get_settings
settings = get_settings()

settings


## 1) Inicializar clientes (Gemini + Pinecone)

In [ ]:
from natubot_core.gemini_client import GeminiClient
from natubot_core.pinecone_client import PineconeClients

gemini = GeminiClient(
    api_key=settings.gemini_api_key,
    chat_model=settings.gemini_chat_model,
    embed_model=settings.gemini_embed_model,
    embed_dim=settings.embed_dim,
)

pinecone = PineconeClients(
    api_key=settings.pinecone_api_key,
    index_name=settings.pinecone_index_name,
    index_host=settings.pinecone_index_host,
)

print("OK: clientes inicializados")


## 2) Health check del índice / namespace

In [ ]:
stats = pinecone.stats(namespace=settings.pinecone_namespace)
stats


## 3) Recuperar contexto (top_k) y ver matches

In [ ]:
from natubot_core.rag import retrieve_context

question = "¿Para qué sirve este producto y cómo se usa?"
top_k = 5

contexts, citations = retrieve_context(
    question=question,
    gemini=gemini,
    pinecone=pinecone,
    namespace=settings.pinecone_namespace,
    top_k=top_k,
    pinecone_filter=None,
)

len(contexts), citations[:2]


### Ver matches (producto / sección / score)

In [ ]:
for c in contexts:
    md = c["metadata"]
    print(f"- score={c['score']:.4f} | product={md.get('product_name')} | section={md.get('section')} | id={c['id']}")


## 4) Construir prompt final (con evidencia)

In [ ]:
from natubot_core.prompts import build_prompt

prompt = build_prompt(question, contexts)
print(prompt[:2500])  # imprime los primeros caracteres para auditar


## 5) Generar respuesta con Gemini (basada en evidencia)

In [ ]:
answer = gemini.generate(prompt, temperature=0.2, max_output_tokens=800)
answer


## 6) Resultado final (answer + citations)

In [ ]:
result = {
    "answer": answer,
    "citations": citations,
    "used_context": bool(contexts),
}
result


## 7) (Opcional) Cómo probar FastAPI cuando lo estés corriendo

In [ ]:
# Requiere: pip install requests
# import requests
# payload = {"message": "¿Para qué sirve y cómo se usa?", "accepted_terms": True, "top_k": 5}
# r = requests.post("http://localhost:8000/chat", json=payload, timeout=60)
# r.status_code, r.json()
